This notebook is for exploring the use of multivaraible logistic regression inspired by EHJ on LA strain for AF currence post-ablation

## Use StatsModels Logit 

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

from hrs_23_figures import *

from sklearn import metrics

regions=['global', 'roof', 'sept', 'lat', 'ant', 'post']

from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
from sklearn.linear_model import LogisticRegression


In [3]:
path="/home/csi20/Dropbox/phd/Data/kiru_demographics_df_omitAFimaging_linearStrain.csv"

In [4]:
cases_df = pd.read_csv(path)

In [5]:
cases_df.columns

Index(['Case', 'Age', 'Sex', 'NYHA class', 'Device present?', 'AF',
       'pre ECG rhythm', 'Pacing changes for CT', 'Pacing',
       'Most recent pacing check prior to CT', 'CT heart rhythm', 'MS QRSd',
       'AL QRSd', 'Comments', 'CT date', 'Heart rate', 'LA strain',
       'LA volume', 'LA size', 'LA emptying fraction', 'LA pressure',
       'LV ES volume', 'LVES diameter (cm)', 'LVEF', 'MR/severity', 'HF meds',
       'Previous ablation', 'Prior mitral procedures', 'af_num', 'area_global',
       'area_roof', 'area_sept', 'area_lat', 'area_ant', 'area_post',
       'area_global_mshqual', 'area_roof_mshqual', 'area_sept_mshqual',
       'area_lat_mshqual', 'area_ant_mshqual', 'area_post_mshqual',
       'f1_global', 'f1_roof', 'f1_sept', 'f1_lat', 'f1_ant', 'f1_post',
       'f2_global', 'f2_roof', 'f2_sept', 'f2_lat', 'f2_ant', 'f2_post',
       'LA_vol_CT', 'LAEF_CT', 'Sex_num', 'LV_vol_CT', 'LVEF_CT',
       'area_global_vxm', 'area_roof_vxm', 'area_sept_vxm', 'area_lat_vxm',


In [57]:
cases_df['Sex_num']

0     1.0
1     1.0
2     0.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     0.0
9     1.0
10    1.0
11    1.0
12    1.0
13    0.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    0.0
28    1.0
29    0.0
Name: Sex_num, dtype: float64

In [58]:
## variables to test univariate regression with AF incidence

list_of_input_vars = ['Age', 'LVEF_CT', 'Sex_num', 'LAEF_CT', 'LA_vol_CT', 'area_global', 'area_roof', 'area_sept',
                      'area_lat', 'area_ant', 'area_post',
                     'f1_global_linear', 'f1_roof_linear', 'f1_sept_linear',
                      'f1_lat_linear', 'f1_ant_linear', 'f1_post_linear']

In [9]:
def logistic_regression(list_of_input_vars, y):
    odds_ratios = pd.DataFrame()
    for variable in list_of_input_vars:
        ## Load and scale input variables
        X = cases_df[[variable]]
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        ## Fit logistic regression model
        mod = sm.Logit(y, X_scaled)
        res = mod.fit()

        data = {"OR": res.params, "Lower CI": res.conf_int()[0], "Upper CI": res.conf_int()[1]}
        odds_ratios = odds_ratios.append(pd.DataFrame(data=data), ignore_index=True)
        odds_ratios = np.exp(odds_ratios)
        odds_ratios['p value'] = res.pvalues[0]
        odds_ratios.rename(index={'x1': variable})
    return odds_ratios

In [11]:
odds_ratios = logistic_regression(list_of_input_vars, y)
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.511386
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.499729
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.540432
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.644091
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.592995
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.597493
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.576408
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.524933
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.537251
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.618599
  

/home/csi20/anaconda3/envs/vtk/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: overflow encountered in exp
  result = func(self.values, **kwargs)
/home/csi20/anaconda3/envs/vtk/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: overflow encountered in exp
  result = func(self.values, **kwargs)
/home/csi20/anaconda3/envs/vtk/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: overflow encountered in exp
  result = func(self.values, **kwargs)
/home/csi20/anaconda3/envs/vtk/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: overflow encountered in exp
  result = func(self.values, **kwargs)
/home/csi20/anaconda3/envs/vtk/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: overflow encountered in exp
  result = func(self.values, **kwargs)
/home/csi20/anaconda3/envs/vtk/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWa

In [15]:
data

{'OR': x1   -1.361984
 dtype: float64,
 'Lower CI': x1   -2.452164
 Name: 0, dtype: float64,
 'Upper CI': x1   -0.271805
 Name: 1, dtype: float64}

In [22]:
res.conf_int()[0][0]

-2.4521636872179657

## Results of Univariate Regression

In [59]:
y = cases_df['af_num']
## Initialise empty dataframe
final_df = pd.DataFrame(columns=['Variable','OR',"Lower CI", "Upper CI", "p-value"])

for variable in list_of_input_vars:
    ## Load and scale input variables
    X = cases_df[[variable]]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    ## Fit logistic regression model
    mod = sm.Logit(y, X_scaled)
    res = mod.fit()
    
    new_row = {"Variable": variable, "OR": np.exp(res.params[0]), "Lower CI": np.exp(res.conf_int()[0][0]),
               "Upper CI": np.exp(res.conf_int()[1][0]), "p-value": res.pvalues[0]}
#     print(new_row)
#     data = np.exp(data)
    
#     odds_ratios = pd.DataFrame(data=data)
    
#     odds_ratios = np.exp(odds_ratios)
#     odds_ratios['p value'] = res.pvalues[0]
    
#     odds_ratios.rename(index={'x1': variable})
    
    final_df = final_df.append(new_row, ignore_index=True)
    
print(final_df, "\n")

Optimization terminated successfully.
         Current function value: 0.679177
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.680130
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.689125
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.511386
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.499729
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.540432
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.644091
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.592995
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.597493
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.576408
  

In [45]:
cases_df.columns

Index(['Case', 'Age', 'Sex', 'NYHA class', 'Device present?', 'AF',
       'pre ECG rhythm', 'Pacing changes for CT', 'Pacing',
       'Most recent pacing check prior to CT', 'CT heart rhythm', 'MS QRSd',
       'AL QRSd', 'Comments', 'CT date', 'Heart rate', 'LA strain',
       'LA volume', 'LA size', 'LA emptying fraction', 'LA pressure',
       'LV ES volume', 'LVES diameter (cm)', 'LVEF', 'MR/severity', 'HF meds',
       'Previous ablation', 'Prior mitral procedures', 'af_num', 'area_global',
       'area_roof', 'area_sept', 'area_lat', 'area_ant', 'area_post',
       'area_global_mshqual', 'area_roof_mshqual', 'area_sept_mshqual',
       'area_lat_mshqual', 'area_ant_mshqual', 'area_post_mshqual',
       'f1_global', 'f1_roof', 'f1_sept', 'f1_lat', 'f1_ant', 'f1_post',
       'f2_global', 'f2_roof', 'f2_sept', 'f2_lat', 'f2_ant', 'f2_post',
       'LA_vol_CT', 'LAEF_CT', 'Sex_num', 'LV_vol_CT', 'LVEF_CT',
       'area_global_vxm', 'area_roof_vxm', 'area_sept_vxm', 'area_lat_vxm',


In [46]:
cases_df['NYHA class']

0     2
1     3
2     3
3     3
4     3
5     3
6     2
7     3
8     2
9     2
10    3
11    2
12    3
13    2
14    3
15    2
16    2
17    2
18    2
19    3
20    3
21    3
22    3
23    2
24    3
25    3
26    3
27    2
28    3
29    3
Name: NYHA class, dtype: int64

In [48]:
af_df = cases_df[cases_df['af_num']==1]
naf_df = cases_df[cases_df['af_num']==0]

In [49]:
stats.ttest_ind(af_df['NYHA class'], naf_df['NYHA class'])

Ttest_indResult(statistic=-1.1245070604634968, pvalue=0.2703460386162905)

## Multivariate Regression

In [64]:
X

,Age,LVEF_CT,Sex_num,LAEF_CT,LA_vol_CT,area_global,area_roof,area_sept,area_lat,area_ant,area_post,f1_global_linear,f1_roof_linear,f1_sept_linear,f1_lat_linear,f1_ant_linear,f1_post_linear
0,83,34.710117,1.0,9.458650,113.025241,10.744721,3.658609,16.252890,25.681326,9.761136,15.363472,3.593031,1.676428,10.187199,8.115237,4.894997,4.398445
1,67,20.378837,1.0,24.386233,123.119015,14.608819,6.173769,17.241993,29.346305,16.128061,20.023663,8.719735,4.867207,15.360185,11.847472,11.926033,5.780665
2,79,49.405136,0.0,48.816523,58.764411,46.434490,15.634638,45.729316,62.065586,48.309589,70.133219,21.276454,9.157319,23.407006,25.349602,27.005228,25.813103
3,50,37.162082,1.0,48.932599,43.064447,35.016327,17.210376,58.557527,53.011690,54.538258,29.571379,16.404347,8.352888,23.146774,26.109833,27.359347,13.262588
4,63,25.947644,1.0,41.992833,64.178693,41.074329,34.873464,48.516791,40.255159,40.350298,45.977682,29.554456,26.408858,33.397165,40.662166,32.634935,26.506252
5,85,27.336696,1.0,22.260599,135.001487,16.397361,5.775808,24.046177,27.902677,17.005839,16.489278,5.919388,3.367325,9.782614,9.266151,9.736088,2.108552
6,38,38.350474,1.0,18.605451,129.492727,17.937271,4.732478,25.745352,40.924687,14.891488,25.975715,6.443928,4.088990,10.248580,8.104541,7.908384,6.684386
7,84,47.518920,1.0,16.338292,132.010091,25.235229,14.731017,33.763684,41.234328,29.523971,26.130387,14.505299,10.626404,23.471893,22.478673,15.184839,17.343903
8,69,19.981491,0.0,24.826958,117.872861,13.285606,10.891380,19.647147,24.869612,11.881721,7.892779,8.915509,6.948727,17.327106,9.226418,9.697387,4.820620
9,79,14.275876,1.0,2.309371,142.974516,7.745421,19.653687,18.723967,36.322877,18.851106,11.276315,13.824883,22.674792,23.293289,28.884781,12.804389,16.542445


In [72]:
final_df = pd.DataFrame(columns=['Variable','OR',"Lower CI", "Upper CI", "p-value"])

# X = cases_df[["LA_vol_CT", "area_post"]]
X = cases_df[['LA_vol_CT', 'f1_global_linear']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## Fit logistic regression model
mod = sm.Logit(y, X_scaled)
res = mod.fit()

print(res.summary())

np.exp(res.params)

# {"Variable": variable, "OR": np.exp(res.params[0]), "Lower CI": np.exp(res.conf_int()[0][0]),
#                "Upper CI": np.exp(res.conf_int()[1][0]), "p-value": res.pvalues[0]}

Optimization terminated successfully.
         Current function value: 0.479988
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 af_num   No. Observations:                   30
Model:                          Logit   Df Residuals:                       28
Method:                           MLE   Df Model:                            1
Date:                Wed, 12 Jul 2023   Pseudo R-squ.:                  0.2142
Time:                        10:53:00   Log-Likelihood:                -14.400
converged:                       True   LL-Null:                       -18.326
Covariance Type:            nonrobust   LLR p-value:                  0.005075
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.1531      0.693      1.664      0.096      -0.205       2.511
x2            -0.7232      0.

x1    3.168155
x2    0.485186
dtype: float64

## Type of AF: Sub-analysis

In [77]:
paro_df = cases_df[cases_df['AF']=='paroxysmal']
pers_df = cases_df[cases_df['AF']=='persistent']

In [79]:
paro_df.columns

Index(['Case', 'Age', 'Sex', 'NYHA class', 'Device present?', 'AF',
       'pre ECG rhythm', 'Pacing changes for CT', 'Pacing',
       'Most recent pacing check prior to CT', 'CT heart rhythm', 'MS QRSd',
       'AL QRSd', 'Comments', 'CT date', 'Heart rate', 'LA strain',
       'LA volume', 'LA size', 'LA emptying fraction', 'LA pressure',
       'LV ES volume', 'LVES diameter (cm)', 'LVEF', 'MR/severity', 'HF meds',
       'Previous ablation', 'Prior mitral procedures', 'af_num', 'area_global',
       'area_roof', 'area_sept', 'area_lat', 'area_ant', 'area_post',
       'area_global_mshqual', 'area_roof_mshqual', 'area_sept_mshqual',
       'area_lat_mshqual', 'area_ant_mshqual', 'area_post_mshqual',
       'f1_global', 'f1_roof', 'f1_sept', 'f1_lat', 'f1_ant', 'f1_post',
       'f2_global', 'f2_roof', 'f2_sept', 'f2_lat', 'f2_ant', 'f2_post',
       'LA_vol_CT', 'LAEF_CT', 'Sex_num', 'LV_vol_CT', 'LVEF_CT',
       'area_global_vxm', 'area_roof_vxm', 'area_sept_vxm', 'area_lat_vxm',


In [82]:
for i in regions:
    print(i)
    
    ans = stats.ttest_ind(paro_df[f'area_{i}'], pers_df[f'area_{i}'])
    
#     ans = stats.ttest_ind(paro_df[f'f1_{i}_linear'], pers_df[f'f1_{i}_linear'])
    print(ans)

global
Ttest_indResult(statistic=-0.030122287681115214, pvalue=0.9768103612818478)
roof
Ttest_indResult(statistic=1.4729077392317875, pvalue=0.18426125481322372)
sept
Ttest_indResult(statistic=1.5415969974288026, pvalue=0.16707382923874328)
lat
Ttest_indResult(statistic=0.22695028892129482, pvalue=0.8269478401196424)
ant
Ttest_indResult(statistic=1.000493837100735, pvalue=0.35039382455775536)
post
Ttest_indResult(statistic=-0.15361468493194622, pvalue=0.8822482865938184)
